In [1]:
import pandas as pd
from pandas import Series, DataFrame
import country_converter as coco
import string
import os
import numpy as np

In [2]:
path = '/Users/ariana/desktop/historical_tech'
os.chdir(path)

In [3]:
# https://www.usgs.gov/centers/national-minerals-information-center/historical-global-statistics-mineral-and-material


In [4]:
def read_usgs_896(tech_name, file_name, sheet_name=0):
    df = pd.read_excel(file_name, sheet_name=sheet_name, header=3,
                       na_values=['XX','--', 'W', '(3/)', '(4/)','-- '])
    drop_idx = []
    for idx in df.index:
        country = df[df.columns[0]].iloc[idx]
        drop_list = ['Limonite', 'Other', 'Puerto Rico', 'United States: Puerto Rico', 'Leeward and Windward Islands ']
        if country in drop_list:
            drop_idx.append(idx)
    df.drop(index=drop_idx, inplace=True)
    df.reset_index(drop=True, inplace=True)
    for idx in df.index:
        country = df[df.columns[0]].iloc[idx]
        if country == 'World':
            world_idx = idx + 1
    df = df[:world_idx]
    countries = []
    for country in df[df.columns[0]]:
        country = country.strip(string.digits)
        country = country.strip()
        countries.append(country)
    df['Country Name'] = countries
    df.drop(columns=df.columns[0], inplace=True)
    iso2 = []
    spatial_scale = []
    for country in df['Country Name']:
        if country == 'Serbia and Montenegro':
            iso2.append('CS')
            spatial_scale.append('National')
        elif country == 'Czechoslovakia':
            iso2.append('CSK')
            spatial_scale.append('National')
        elif country == 'U.S.S.R.':
            iso2.append('SU')
            spatial_scale.append('National')
        elif country == 'Yugoslavia':
            iso2.append('YU')
            spatial_scale.append('National')
        elif country == 'World':
            iso2.append('World')
            spatial_scale.append('Global')
        elif country == 'Netherlands Antilles':
            iso2.append('AN')
            spatial_scale.append('National')
        else:
            iso2.append(coco.convert(names=country, to='iso2'))
            spatial_scale.append('National')
    df['Country Code'] = iso2
    df['Spatial Scale'] = spatial_scale
    df['Data Source'] = 'U.S. Geological Survey'
    df['Unit'] = 'Metric tons'
    df['Metric'] = 'Annual production'
    df['Technology Name'] = tech_name
    df['ID'] = df['Technology Name'] + '_' + df['Metric'] + '_' + df['Country Code']
    df.set_index('ID', inplace=True)
    omit = []
    for col in df.columns:
        if len(str(col)) > 4 or col == 'Unit':
            omit.append(col)
    empty_rows = df.drop(columns=omit)
    empty_rows.dropna(how='all', inplace=True)
    na_idx = []
    for country in df.index:
        if country not in empty_rows.index:
            na_idx.append(country)
    df.drop(na_idx, inplace=True)
    file_name = 'cleaned data/ds896_' + tech_name + '.csv'
    df.to_csv(file_name)
    return df


In [5]:
# https://www.usgs.gov/centers/national-minerals-information-center/historical-global-statistics-mineral-and-material#:~:text=Aluminum-,ds896%2Daluminum.xlsx,-1990%2D2012
# Aluminum and Bauxite


In [6]:
aluminum = read_usgs_896('Primary aluminum production', 'raw data/usgs/ds896-aluminum.xlsx', 'Aluminum, primary production')

/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [7]:
bauxite = read_usgs_896('Primary bauxite production', 'raw data/usgs/ds896-aluminum.xlsx', 'Bauxite')


/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [8]:
# https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/mineral-pubs/historical-statistics/global/ds896-cadmium.xlsx
# Cadmium


In [9]:
# omit East/West Germany since there's only one year
cadmium = read_usgs_896('Cadmium Refining', 'raw data/usgs/ds896-cadmium.xlsx')
cadmium = cadmium[(cadmium['Country Name']!='Germany: Eastern states') & (cadmium['Country Name']!='Germany: Western states')]
cadmium.to_csv('cleaned data/ds896_Cadmium Refining.csv')


/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [10]:
# https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/mineral-pubs/historical-statistics/global/ds896-copper.xlsx
# Copper mining and refining


In [11]:
# check for str
copper_mining = read_usgs_896('Copper Mining', 'raw data/usgs/ds896-copper.xlsx', 'Mine')
copper_mining = copper_mining[copper_mining['    Mine production']=='Mine: Total']
copper_mining.drop(columns='    Mine production', inplace=True)
copper_mining.to_csv('cleaned data/ds896_Copper Mining.csv')

In [12]:
# check for str
copper_refining = read_usgs_896('Copper Refining', 'raw data/usgs/ds896-copper.xlsx', 'Refinery')
copper_refining = copper_refining[copper_refining['Refinery production']=='Refinery: Total']
copper_refining.drop(columns='Refinery production', inplace=True)
copper_refining.to_csv('cleaned data/ds896_Copper Refining.csv')

In [13]:
# https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/mineral-pubs/historical-statistics/global/ds896-iron-steel.xlsx
# Iron ore and raw steel


In [14]:
# omit West Germany bc only one data point
iron = read_usgs_896('Iron ore', 'raw data/usgs/ds896-iron-steel.xlsx', 'Iron ore, gross weight')
iron = iron[(iron['Country Name']!='Germany: Western states')]
iron.to_csv('cleaned data/ds896_Iron ore.csv')

/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [15]:
# check for str
steel = read_usgs_896('Raw steel production', 'raw data/usgs/ds896-iron-steel.xlsx', 'Raw steel')
steel = steel[(steel['Country Name']!='Germany: Eastern states') & (steel['Country Name']!='Germany: Western states')]
steel.to_csv('cleaned data/ds896_Raw steel production.csv')

/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [16]:
# https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/mineral-pubs/historical-statistics/global/ds896-lead.xlsx
# Lead


In [17]:
lead = read_usgs_896('Lead mine production', 'raw data/usgs/ds896-lead.xlsx', 'Mine')
lead = lead[lead['Country Name']!='Germany: Western states']
lead.to_csv('cleaned data/ds896_Lead mine production.csv')

/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [18]:
# https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/mineral-pubs/historical-statistics/global/ds896-salt.xlsx
# Salt


In [19]:
salt = pd.read_excel('raw data/usgs/ds896-salt.xlsx', sheet_name=0, header=3,
                     na_values=['XX','--', 'W', '(3/)', '(4/)','-- '])
drop_idx = []
for idx in salt.index:
    country = salt[salt.columns[0]].iloc[idx]
    drop_list = ['Limonite', 'Other', 'Puerto Rico', 'United States: Puerto Rico', 'Leeward and Windward Islands ']
    if country in drop_list:
        drop_idx.append(idx)
salt.drop(index=drop_idx, inplace=True)
salt.reset_index(drop=True, inplace=True)
for idx in salt.index:
    country = salt[salt.columns[0]].iloc[idx]
    if country == 'World':
        world_idx = idx + 1
salt = salt[:world_idx]
countries = []
for country in salt[salt.columns[0]]:
    country = country.strip(string.digits)
    country = country.strip()
    countries.append(country)
salt['Country Name'] = countries
salt = salt[(salt['Country Name']!='Germany: Eastern states') & (salt['Country Name']!='Germany: Western states')]
salt.drop(columns=[salt.columns[0], 'Salt type'], inplace=True)
unique_countries = sorted(list(set(salt['Country Name'])))
salt_type_sum = []
for country in unique_countries:
    dup_df = salt[salt['Country Name']==country].sum()
    dup_df = dup_df[:-1]
    val_list = []
    for x in dup_df:
        val_list.append(x)
    val_list.append(country)
    salt_type_sum.append(val_list)
salt = pd.DataFrame(salt_type_sum, columns=salt.columns)
iso2 = []
spatial_scale = []
for country in salt['Country Name']:
    if country == 'Serbia and Montenegro':
        iso2.append('CS')
        spatial_scale.append('National')
    elif country == 'Czechoslovakia':
        iso2.append('CSK')
        spatial_scale.append('National')
    elif country == 'U.S.S.R.':
        iso2.append('SU')
        spatial_scale.append('National')
    elif country == 'Yugoslavia':
        iso2.append('YU')
        spatial_scale.append('National')
    elif country == 'World':
        iso2.append('World')
        spatial_scale.append('Global')
    elif country == 'Netherlands Antilles':
        iso2.append('AN')
        spatial_scale.append('National')
    else:
        iso2.append(coco.convert(names=country, to='iso2'))
        spatial_scale.append('National')
salt['Country Code'] = iso2
salt['Spatial Scale'] = spatial_scale
salt['Data Source'] = 'U.S. Geological Survey'
salt['Unit'] = 'Metric tons'
salt['Metric'] = 'Annual production'
salt['Technology Name'] = 'Salt production'
salt['ID'] = salt['Technology Name'] + '_' + salt['Metric'] + '_' + salt['Country Code']
salt.set_index('ID', inplace=True)
salt.replace(0, np.nan, inplace=True)
salt.to_csv('cleaned data/ds896_Salt production.csv')


In [20]:
# https://www.usgs.gov/centers/national-minerals-information-center/historical-statistics-mineral-and-material-commodities
# Data Series 140


In [21]:
def read_usgs_140(tech_name, file_name, sheet_name=0, 
                  col_names=['Year','Production','Unit value (98$/t)','World production']):
    df = pd.read_excel(file_name, header=4, usecols=col_names, na_values=['XX','--', 'W', '(3/)', '(4/)','-- '])
    end_idx = None
    for idx in df.index:
        year = df['Year'].iloc[idx]
        if type(year) != int:
            end_idx = idx
            break
    df = df[:end_idx]
    df.set_index('Year', drop=True, inplace=True)
    df = df.transpose()
    df['Technology Name'] = tech_name
    df['Data Source'] = 'U.S. Geological Survey'
    df['Country Name'] = ['United States', 'World', 'World']
    df['Country Code'] = ['US', 'World', 'World']
    df['Spatial Scale'] = ['National', 'Global', 'Global']
    df['Unit'] = ['Metric tons', '1998 USD/metric ton', 'Metric tons']
    df['Metric'] = ['Annual production', 'Price', 'Annual production']
    df['ID'] = df['Technology Name'] + '_' + df['Metric'] + '_' + df['Country Code']
    df.set_index('ID', drop=True, inplace=True)
    df.columns.name = None
    file_name = 'cleaned data/ds140_' + tech_name + '.csv'
    df.to_csv(file_name)
    return df


In [22]:
# https://www.usgs.gov/media/files/cement-historical-statistics-data-series-140
# Cement

In [23]:
cement = read_usgs_140('Cement production', 'raw data/usgs/ds140-cement-2019.xlsx')

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [24]:
# https://www.usgs.gov/media/files/copper-historical-statistics-data-series-140
# Primary copper

In [25]:
copper = read_usgs_140('Primary copper production', 'raw data/usgs/ds140-copper-2018.xlsx', 
             col_names=['Year','Primary production','Unit value (98$/t)','World production'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [26]:
# https://www.usgs.gov/media/files/gold-historical-statistics-data-series-140
# Gold

In [27]:
gold = read_usgs_140('Gold production', 'raw data/usgs/ds140-gold-2018.xlsx', 
             col_names=['Year','Primary production','Unit value (98$/t)','World production'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [28]:
# https://www.usgs.gov/media/files/nickel-historical-statistics-data-series-140
# Nickel

In [29]:
# check for str
nickel = read_usgs_140('Nickel production', 'raw data/usgs/ds140-nickel-2018_1.xlsx', 
             col_names=['Year','Primary production','Unit value (98$/t)','World production'])

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [30]:
# https://www.usgs.gov/media/files/construction-sand-and-gravel-historical-statistics-data-series-140
# Sand and gravel (construction)


In [31]:
sand_gravel_construction = read_usgs_140('Sand and gravel production (construction)', 
                                         'raw data/usgs/ds140-sand-gravel-construction-2019.xlsx')

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [32]:
# https://www.usgs.gov/media/files/industrial-sand-and-gravel-historical-statistics-data-series-140
# Sand and gravel (industrial)


In [33]:
sand_gravel_industrial = read_usgs_140('Sand and gravel production (industrial)', 
                                         'raw data/usgs/ds140-sand-industrial-2019.xlsx')

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
